In [ ]:
using Laplacians
using SparseArrays
using LinearAlgebra
using Statistics

In [ ]:
function LDLsolverNC(ldli::Laplacians.LDLinv, b::Vector)
    y = copy(b)

    Laplacians.forward!(ldli, y)

    @inbounds for i in 1:(length(ldli.d))
        if ldli.d[i] != 0
            y[i] /= ldli.d[i]
        end
    end

    Laplacians.backward!(ldli, y)

    # mu = mean(y)
    # @inbounds for i in eachindex(y)
    #     y[i] = y[i] - mu
    # end

    return y
end

In [ ]:
function LDLsolverNC(ldli::Laplacians.LDLinv, Y::SparseMatrixCSC) 
    B = copy(Matrix(Y))
    d, n = size(Y)
    for i in 1:n
        B[:,i] = LDLsolverNC(ldli, B[:,i])
        mu = mean(B[:,i])
    end
    return B
end

function LDLsolver(ldli::Laplacians.LDLinv, Y::SparseMatrixCSC) 
    B = copy(Matrix(Y))
    d, n = size(Y)
    for i in 1:n
        B[:,i] = Laplacians.LDLsolver(ldli, B[:,i])
        mu = mean(B[:,i])
    end
    return B
end

In [ ]:
n = 6
a = grid2(3,4)

In [ ]:
ldliLapA = zeros(eltype(a), size(a))
num_iter = 1
spli_merge=1000000
for i in range(1, num_iter)
    llmat = Laplacians.LLmatp(a,spli_merge);
    ldli = Laplacians.approxChol(llmat,spli_merge,spli_merge);
    ldliLapA += LDLsolverNC(ldli, lap(a));
end
ldliLapA = ldliLapA ./ num_iter

In [ ]:
abs.(ldliLapA) .> 1e-1

In [ ]:
ldliLapAOrd = zeros(eltype(a), size(a))
num_iter = 10000
for i in range(1, num_iter)
    llmatOrd = Laplacians.LLMatOrd(a);
    @time ldliOrd = Laplacians.approxChol(llmatOrd);
    ldliLapAOrd += LDLsolverNC(ldliOrd, lap(a));
end
ldliLapAOrd = ldliLapAOrd ./ num_iter;

In [ ]:
abs.(ldliLapAOrd) .> 1e-1